In [146]:
import pandas as ps
from pandas import DataFrame 
import json
import re

In [83]:
output = ps.read_csv("./website/output_rag_for_website.csv")
print(output.columns)

Index(['Unnamed: 0', 'word', 'genre', 'complete_response', 'generated_text',
       'answer_choices', 'probability', 'full_word_tokenized', 'is_valid'],
      dtype='object')


In [84]:
semantic_eval = ps.read_csv("./website/semantic_eval_1000_word_genre.csv")
print(semantic_eval.columns)

Index(['genre', 'word', 'similarity_score', 'definition', 'answer_choices'], dtype='object')


In [85]:
word_def = ps.read_csv("./website/sat_words_with_definition.csv")
print(word_def.head(1))

    word part_of_speech                                         definition
0  Abate              v  to reduce in amount, degree, or severity; The ...


In [155]:
data = []
for index, row in output.iterrows():
    word = row['word']
    genre = row['genre']
    try:
        question = row['generated_text'].strip()
        
        # if word not included, ignore the response
        if question.find(" ________________ ") == -1:
            continue

        #print("Before: {}".format(question))
        question = re.sub(r'\r?\n',' ',question)
        question = re.sub(r' +',' ',question).strip()

        # format question
        if question.find("The paragraph should be") != -1:
            sp = question.split('.')
            sp.pop(0)
            question = ".".join(sp)
        
        if question.find("Here is the paragraph:") != -1:
            sp = question.split(':')
            sp.pop(0)
            question = ":".join(sp)

        #print(question.find("Here is the paragraph:"))
        #print("After: {}".format(question))
        
        definition = word_def.query('word.str.lower() == "{}"'.format(word))['definition'].item()
    
        # format prompt
        prompt = '''Given the meaning of the word {}: {}, Generate a paragraph on {} with the word {}. Make sure your paragraph is one single paragraph that is formally worded. If you are done generating the paragraph, stop. Make sure to use the given word "{}" as is and only use the word "{}" once. The paragraph should be a formal description of {}.'''.format(word, definition, genre, word, word, word, genre)

        answer_choices = row['answer_choices'].split(",")
        answer_choices_with_def={}
        for i in range (len(answer_choices)):
            choice = answer_choices[i]
            result = word_def.query('word.str.lower() == "{}"'.format(choice))
            choice_def = result['definition'].values[0]
            answer_choices_with_def[choice] = choice_def  
        #print(answer_choices_with_def)
    
        score = semantic_eval.query('word.str.lower() == "{}" and genre.str.lower() == "{}"'.format(word, genre))
        semantic_score = score['similarity_score'].values[0]
        
        row_output = {
            "word": word,
            "genre": genre, 
            "semantic_score": semantic_score,
            "prompt": prompt, 
            "question": question, 
            "ans_choices": answer_choices,
            "ans_choices_def":answer_choices_with_def, 
            "probability":row['probability'],  
            "tokenization":row['full_word_tokenized'], 
            "is_valid":row['is_valid']
        }
        
        data.append(json.dumps(row_output))

    except Exception as err:
        print(Exception, err)



<class 'Exception'> index 0 is out of bounds for axis 0 with size 0
<class 'Exception'> invalid syntax (<unknown>, line 1)
<class 'Exception'> 'float' object has no attribute 'strip'
<class 'Exception'> index 0 is out of bounds for axis 0 with size 0
<class 'Exception'> invalid syntax (<unknown>, line 1)


In [156]:
print(len(data))

835


In [157]:
with open("./website/questions.jsonl", "w") as f:
    for i in range(len(data)):
        f.write(data[i])
        f.write('\n')